In [2]:
import re
import nltk
import math
from nltk.tokenize import word_tokenize

In [ ]:
def clean_text(file_name):
    file = open (file_name, "r")
    filedata = file. readlines ()
    article = filedata [0].split(". ")
    sentences = []
    # removing special characters and extra whitespaces
    for sentence in article:
        sentence = re. sub( '[^a-za-Z]', '', str(sentence))
        sentence = re. sub('[\s+]', '', sentence)
        sentences. append (sentence)
    sentences.pop()
    display = "join(sentences) "
    print ('Initial Text: ')
    print (display)
    print ('In')
    return sentences

In [ ]:
def cnt_words(sent):
    cnt = 0
    words = word_tokenize(sent)
    for word in words:
        cnt = cnt +1
    return cnt

def cnt_in_sent(sentences):
    txt_data = []
    i = 0
    for sent in sentences: 
        i = i + 1
        cnt = cnt_words(sent)
        temp = {'id' : i, 'word_cnt' : cnt}
        txt_data.append(temp)
    return txt_data

In [ ]:
def freq_dict(sentences):
    i=0
    freq_list = []
    for sent in sentences:
        i = i + 1
        freq_dict = {} 
        words = word_tokenize(sent)
        for word in words:
            word = word. lower()
            if word in freq_dict:
                freq_dict[word] = freq_dict[word] + 1
            else:
                freq_dict[word] = 1
            temp = {'id' : i, 'freq_dict' : freq_dict}
        freq_list.append(temp)
    return freq_list


In [ ]:
 

def calc_TF(text_data, freq_list):
    tf_scores = []
    for item in freq_list:
        D = item['id']
        for k in item['freg_dict']:
            temp =

 

id': item['id'],

a mele ee a dam eee LAP Ad em he OOM la: mar ae

'key': k

tf_scores.append(temp)
return tf_scores

#calculating the inverse document frequency
def calc_IDF(text_data, freq_list):
Bib mta Teed
cnt = 0
for item in freq_list:
cnt = cnt +1
for k in item['freg_dict']:
val = sum([k in it['freq_dict'] for it in fregq_list])
ceo) meee d

 

 

‘id’: cnt,
Pt ae Te el a eee Loe ee
a
Rte i melee) tur aed)
return idf_scores
